## Initialization

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter, OrderedDict
from dataclasses import dataclass
from pathlib import Path
import os
import pickle
import random
import math
import numpy as np

from typing import Union, Optional
from types import MethodType

import matplotlib.pyplot as plt
import plotly.express as px
    
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader, Sampler, BatchSampler
from torch.distributions import Dirichlet, Categorical
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import tqdm
import wandb

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from pathlib import Path
root_dir = Path("/mnt/data/factcheck/tweets/clustering/")

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## VGG

In [3]:
# from https://github.com/GT-RIPL/L2C/blob/master/models/vgg.py

class VGG(nn.Module):
    cfg = {
        'S': [64, 'M', 128, 'M', 256, 'M', 256, 'M'],
        8: [64, 'M', 128, 'M', 256, 'M', 512, 'M', 512, 'M'],
        11: [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        13: [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        16: [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
        19: [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    }

    def __init__(self, n_layer, out_dim=10, in_channels=3, img_sz=32):
        super(VGG, self).__init__()
        self.conv_func = nn.Conv2d
        self.features = self._make_layers(VGG.cfg[n_layer],in_channels)
        if n_layer=='S':
            self.feat_map_sz = img_sz // 16
            feat_dim = 256*(self.feat_map_sz**2)
            self.last = nn.Sequential(
                nn.Linear(feat_dim, feat_dim//2),
                nn.BatchNorm1d(feat_dim//2),
                nn.ReLU(inplace=True),
                nn.Linear(feat_dim//2, out_dim)
            )
            self.last.in_features = feat_dim
        else:
            self.feat_map_sz = img_sz // 32
            self.last = nn.Linear(512*(self.feat_map_sz**2), out_dim)

    def _make_layers(self, cfg, in_channels):
        layers = []
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)

    def logits(self, x):
        x = self.last(x)
        return x

    def forward(self, x, pair_enum=True):
        x = self.features(x)
        x = self.logits(x.view(x.size(0), -1))
        return x


def VGGS(out_dim):
    return VGG(n_layer='S', out_dim=out_dim, in_channels=1)

## Arguments

In [4]:
@dataclass
class ArgsOmniglotDAC:
    model_type: str = 'dac'

    lr: float = 1e-4
    
    partial_fraction: Optional[float] = None

    feature_size: int = 1024  # number of features extracted from convolutional layers
    dim: int = 256
    num_blocks: int = 2  # Number of stacks of SABs/ISABs
    num_heads: int = 4
    num_inds: int = 32  # Output dimension of ISABs/PMAs

    drop_p: float = 0.1  # Dropout
    ln: bool = False  # Layer normalization

In [4]:
args = ArgsOmniglotDAC()

## Rand Subclass Sampler and AverageMeter

In [5]:
# from https://github.com/GT-RIPL/L2C/blob/master/dataloaders/sampler.py
class RandSubClassSampler(torch.utils.data.sampler.Sampler):
    r"""Samples a subset of classes for each mini-batch, without replacement.
    Arguments:
        inds (list): a list of indices
        labels (list): a list of class labels
        cls_per_batch (int): number of class in each mini-batch
        batch_size (int): mini-batch size
        num_batch (int): number of mini-batch
        ADDED: rng (RandomSeed or None)
    """

    def __init__(self, inds, labels, cls_per_batch, batch_size, num_batch, seed=None):
        assert len(inds) == len(labels), "Mismatched inputs inds:%d,labels:%d" % (
            len(inds), len(labels))
        self.batch_size = batch_size
        self.cls_per_batch = cls_per_batch
        self.num_batch = num_batch
        self.sample_per_cls = batch_size//cls_per_batch
        self.inds = inds
        self.labels = labels
        self.cls2ind = {}
        self.label_set = list(set(labels))
        self.seed = seed

        for l in self.label_set:
            self.cls2ind[l] = []
        for i in range(len(inds)):
            self.cls2ind[labels[i]].append(inds[i])

    def __iter__(self):
        rng = np.random.RandomState(seed=self.seed)
        for b in range(self.num_batch):
            rand_cls_set = rng.choice(
                self.label_set, self.cls_per_batch, replace=False)
            for c in rand_cls_set:
                ind_list = rng.choice(
                    self.cls2ind[c], self.sample_per_cls, replace=False)
                for i in ind_list:
                    yield i

    def __len__(self):
        return self.batch_size * self.num_batch

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Omniglot

In [6]:
def OmniglotL2C(batch_size, bag_size, cls_per_bag=20, num_workers=2, seed=None, test_seed=1234):
    # based on https://github.com/GT-RIPL/L2C/blob/master/dataloaders/default.py
    binary_flip = transforms.Lambda(lambda x: 1 - x)
    normalize = transforms.Normalize((0.086,), (0.235,))
    train_dataset = torchvision.datasets.Omniglot(
        root='data', download=True, background=True,
        transform=transforms.Compose(
            [transforms.RandomResizedCrop(32, (0.85, 1.)),
             transforms.ToTensor(),
             binary_flip,
             normalize]
        ))

    n_batch_elements = bag_size*batch_size
    print(f"n_batch_elements: {n_batch_elements}")
    
    train_length = len(train_dataset)
    train_imgid2cid = [train_dataset[i][1] for i in range(train_length)]  # train_dataset[i] returns (img, cid)
    # Randomly select 20 characters from 964. By default setting (batch_sz=100), each character has 5 images in a mini-batch.    

    train_sampler = RandSubClassSampler(
        inds=range(train_length),
        labels=train_imgid2cid,
        cls_per_batch=cls_per_bag,
        batch_size=bag_size,
        num_batch=train_length // bag_size,
        seed=seed)
    
    def collate(E):
        element_shape = E[0][0].shape
        # actual batch size may be less than batch_size
        X = torch.vstack([e[0] for e in E]).reshape(-1, bag_size, *element_shape)
        T = torch.LongTensor([e[1] for e in E]).reshape(-1, bag_size)
        return X, T
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=n_batch_elements, shuffle=False,
                                               num_workers=num_workers, sampler=train_sampler, collate_fn=collate)
    train_loader.num_classes = 964

    test_dataset = torchvision.datasets.Omniglot(
        root='data', download=True, background=False,
        transform=transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             binary_flip,
             normalize]
        ),
        # let evaluation classes start at 1000 - it is easier for debugging
        target_transform=lambda cls_ids: cls_ids + 1000
    )
    eval_length = len(test_dataset)
    eval_imgid2cid = [test_dataset[i][1] for i in range(eval_length)]
    
    eval_sampler = RandSubClassSampler(
        inds=range(eval_length),
        labels=eval_imgid2cid,
        cls_per_batch=cls_per_bag,
        batch_size=bag_size,
        num_batch=eval_length // bag_size,
        seed=test_seed)
    eval_loader = torch.utils.data.DataLoader(test_dataset, batch_size=n_batch_elements, shuffle=False,
                                              num_workers=num_workers, sampler=eval_sampler, collate_fn=collate)
    eval_loader.num_classes = 659

    return train_loader, eval_loader

In [8]:
train_loader, eval_loader = OmniglotL2C(batch_size=100, bag_size=100, cls_per_bag=20, num_workers=2, seed=None, test_seed=1234)

Files already downloaded and verified
n_batch_elements: 10000
Files already downloaded and verified


## Attention (code by Jakub Monhart)

In [7]:
class AddCompat(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.v = nn.Parameter(torch.Tensor(size)) # importance vector
        bound = 1 / self.v.shape[0]**.5
        self.v.data.uniform_(-bound, bound)

    def forward(self, Q, K):
        Q = Q.unsqueeze(-2)        # (*, m, 1, H)
        K  = K.unsqueeze(-3)           # (*, 1, n, H)
        QK = torch.tanh(Q + K)              # (*, m, n, H)
        A_logits = torch.matmul(QK, self.v) # (*, m, n)
        return A_logits
  
    
class MultiCompat(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
  
    def forward(self, Q, K):
        A_logits = (Q @ K.transpose(1,2)) / math.sqrt(self.dim)
        return A_logits

    
class MAB(nn.Module):
    def __init__(self, dim_X, dim_Y, dim, num_heads=4, ln=False, p=None, compat='multi'):
        super().__init__()
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_X, dim)
        self.fc_k = nn.Linear(dim_Y, dim)
        self.fc_v = nn.Linear(dim_Y, dim)
        self.fc_o = nn.Linear(dim, dim)

        self.ln1 = nn.LayerNorm(dim) if ln else nn.Identity()
        self.ln2 = nn.LayerNorm(dim) if ln else nn.Identity()
        self.dropout1 = nn.Dropout(p=p) if p is not None else nn.Identity()
        self.dropout2 = nn.Dropout(p=p) if p is not None else nn.Identity()
    
        if compat == 'multi':
            self.compat = MultiCompat(dim)
        elif compat == 'add':
            self.compat = AddCompat(dim//num_heads)
        else:
            print(f'Compatibility function {compat} not implemented.')
            return -1

    def forward(self, X, Y, mask=None):
        Q, K, V = self.fc_q(X), self.fc_k(Y), self.fc_v(Y)

        # print(Q.shape) # [1, 32, 256]
        # print(K.shape) # [1, 20, 256]
        # print(V.shape) # [1, 20, 256]
        Q_ = torch.cat(Q.chunk(self.num_heads, -1), 0)
        K_ = torch.cat(K.chunk(self.num_heads, -1), 0)
        V_ = torch.cat(V.chunk(self.num_heads, -1), 0)
        # print([e.shape for e in Q.chunk(self.num_heads, -1)]) # [[1, 32, 64],....
        # assert False

        # Compute compatibility
        A_logits = self.compat(Q_, K_)

        if mask is not None:
            mask = mask.squeeze(-1).unsqueeze(1)
            mask = mask.repeat(self.num_heads, Q.shape[1], 1)
            A_logits.masked_fill_(mask, -100.0)
        A = torch.softmax(A_logits, -1)

        attn = torch.cat((A @ V_).chunk(self.num_heads, 0), -1)
        O = self.ln1(Q + self.dropout1(attn))
        O = self.ln2(O + self.dropout2(F.relu(self.fc_o(O))))
        return O

    
class SAB(nn.Module):
    def __init__(self, dim_X, dim, **kwargs):
        super().__init__()
        self.mab = MAB(dim_X, dim_X, dim, **kwargs)

    def forward(self, X, mask=None):
        return self.mab(X, X, mask=mask)

    
class StackedSAB(nn.Module):
    def __init__(self, dim_X, dim, num_blocks, **kwargs):
        super().__init__()
        self.blocks = nn.ModuleList([SAB(dim_X, dim, **kwargs)])
        for i in range(num_blocks-1):
            self.blocks.append(SAB(dim, dim, **kwargs))
      
    def forward(self, X, mask=None):
        for sab in self.blocks:
            X = sab(X, mask=mask)
        return X


class PMA(nn.Module):
    '''
    If used as pooling:
    - num_inds: should be set to 1 for pooling
    - dim_x: dimension of input
    - dim: dimension of output
    '''
    def __init__(self, dim_X, dim, num_inds, **kwargs):
        super().__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim))
        nn.init.xavier_uniform_(self.I)
        self.mab = MAB(dim, dim_X, dim, **kwargs)

    def forward(self, X, mask=None):
        return self.mab(self.I.repeat(X.shape[0], 1, 1), X, mask=mask)
    
class ISAB(nn.Module):
    def __init__(self, dim_X, dim, num_inds, **kwargs):
        super().__init__()
        self.pma = PMA(dim_X, dim, num_inds, **kwargs)
        self.mab = MAB(dim_X, dim, dim, **kwargs)

    def forward(self, X, mask=None):
        return self.mab(X, self.pma(X, mask=mask))

    
class StackedISAB(nn.Module):
    def __init__(self, dim_X, dim, num_inds, num_blocks, **kwargs):
        super().__init__()
        self.blocks = nn.ModuleList([ISAB(dim_X, dim, num_inds, **kwargs)])
        for i in range(num_blocks-1):
            self.blocks.append(ISAB(dim, dim, num_inds, **kwargs))

    def forward(self, X, mask=None):
        for isab in self.blocks:
            X = isab(X, mask=mask)
        return X

## DAC Module (code by Jakub Monhart)

In [8]:
class DACModel(nn.Module):
    def __init__(self, args=None, input_size=None, dim=None, num_inds=None, num_blocks=None, num_heads=None, drop_p=None, ln=None):
        super().__init__()
        if args is not None:
            input_size = args.input_size
            dim = args.dim
            num_inds = args.num_inds
            num_blocks = args.num_blocks
            num_heads = args.num_heads
            drop_p = args.drop_p
            ln = args.ln

        assert [input_size, dim, num_inds, num_blocks, num_heads, drop_p, ln] != [None] * 7

        self.isab = StackedISAB(input_size, dim, num_inds, num_blocks, p=drop_p, ln=ln, num_heads=num_heads)
        self.mab = MAB(dim, dim, dim, ln=ln, p=drop_p, num_heads=num_heads)
        self.isab2 = StackedISAB(dim, dim, num_inds, num_blocks, p=drop_p, ln=ln, num_heads=num_heads)
        self.fc = nn.Linear(dim, 1)


    def forward(self, X, anchor_idxs, mask=None):
        H_enc = self.isab(X, mask=mask)
        anchors = H_enc[torch.arange(X.shape[0]), anchor_idxs].unsqueeze(1)
        H_enc = self.mab(H_enc, anchors)
        x = self.fc(self.isab2(H_enc, mask=mask))
        return x

## DAC Lightning Module (code by Jan Drchal)

In [9]:
class DACLightning(pl.LightningModule):
    def __init__(self, args, schedule_fn=None, featuremodel=None):
        super().__init__()
        self.args = args
        if featuremodel is None:
            featuremodel = nn.Identity()
        self.schedule_fn = schedule_fn
        self.featuremodel = featuremodel
        self.dacmodel = DACModel(input_size=args.feature_size, dim=args.dim, num_inds=args.num_inds,
                                 num_blocks=args.num_blocks, num_heads=args.num_heads, drop_p=args.drop_p, ln=args.ln)

        self.partial_fraction = args.partial_fraction
        
        self.criterion = self.anchored_loss

        self.save_hyperparameters("args")

    def sample_anchor_idxs(self, B, N, mask=None):
        # uniform distribution over bag elements, TODO try uniform over clusters
        if mask is None:
            # return torch.zeros([B], dtype=torch.int64)
            idxs = torch.randint(N, (B,))
            return idxs
        else:
            mask = mask.view(B, N)
            anchor_idxs = torch.zeros(B, dtype=torch.int64)
            for b in range(B):
                if mask[b].sum() < N:
                    idx_pool = mask[b].bitwise_not().nonzero().view(-1)
                    anchor_idxs[b] = idx_pool[torch.randint(len(idx_pool), [1])]
            return anchor_idxs

    def sample_anchor_idxs_partial(self, T):
        B, N = T.shape[:2]

        idxs = torch.zeros(B, dtype=torch.long)
        for b in range(B):
            # print(f"DACLightning.anchored_loss: T[b] = {T[b]}")
            cls_ids = T[b].unique(sorted=True)
            # print(f"DACLightning.anchored_loss: cls_ids = {cls_ids}")
            last_cls_idx = int(np.ceil(len(cls_ids)*self.partial_fraction))
            # print(f"DACLightning.anchored_loss: last_cls_id = {last_cls_idx}")
            if last_cls_idx > 1:
                cls_id = cls_ids[torch.randint(last_cls_idx, (1,)).item()]
            else:
                cls_id = cls_ids[0]
            # print(f"DACLightning.anchored_loss: cls_id = {cls_id}")
            all_idxs = (T[b] == cls_id).nonzero()
            # print(f"DACLightning.anchored_loss: all_idxs = {all_idxs}")
            sel_idx = all_idxs[torch.randint(len(all_idxs), (1, )).item()].item()    
            idxs[b] = sel_idx
        return idxs


    def anchored_loss(self, logits, anchor_idxs, T):
        # own implementation instead of dac.anchored_cluster_loss
        # logits (batch_size, bag_size)
        # T      (batch_size, bag_size)
        batch_size, bag_size = logits.shape[0:2]
        # print(f"DACLightning.anchored_loss: logits.shape = {logits.shape}")
        anchor_cids = T[torch.arange(batch_size), anchor_idxs] 

        # same shape as logits and T: 1.0 for selected cluster, 0.0 otherwise
        targets = (T == anchor_cids.unsqueeze(-1)).float()

        loss = F.binary_cross_entropy_with_logits(logits, targets)
        return loss


    def forward(self, X, anchor_idxs, mask=None):
        # (batch_size, bag_size, input_dim_1, ....)
        batch_size, bag_size = X.shape[0:2]
        Xre = X.reshape([batch_size*bag_size] + list(X.shape[2:])) # combine all bags in batch for the feature extraction phase
        H = self.featuremodel(Xre)  # (batch_size*bag_size, feature_size)
        H = H.reshape(batch_size, bag_size, -1) # get back batches and bags in them: (batch_size, bag_size, feature_size)
        # H = X  # (batch_size, bag_size, feature_size)

        if mask is not None:
            # mask = (batch_size, bag_size)
            mask = mask.unsqueeze(-1) # this is shape accepted by dac.py: (batch_size, bag_size, 1)
        logits = self.dacmodel(H, anchor_idxs, mask=mask) 
        logits = logits.squeeze(-1) # (batch_size, bag_size)
        return logits

    def forward_loss(self, batch, partial=False):
        # X.shape = (batch_size, bag_size, input_dim_1, ...), T.shape = (batch_size, bag_size)
        X, T = batch
        batch_size, bag_size = X.shape[0], X.shape[1]
        if partial:
            anchor_idxs = self.sample_anchor_idxs_partial(T)
        else:
            anchor_idxs = self.sample_anchor_idxs(batch_size, bag_size)

        logits = self(X, anchor_idxs)
        loss = self.criterion(logits, anchor_idxs, T)
        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward_loss(batch, partial=self.partial_fraction is not None)
        self.log("loss", loss, prog_bar=True,
                 logger=True, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # print("validation_step")
        loss, logits = self.forward_loss(batch)
        X, T = batch
        plabels = self.cluster_anchored(X)
        return {"loss": loss, "plabels": plabels, "targets": T}

    def predict_step(self, batch, batch_idx):
        loss, logits = self.forward_loss(batch)
        X, T = batch
        plabels = self.cluster_anchored(X)
        return plabels

    def cluster_anchored(self, X, max_iter=50):        
        # print(f"DACLightning.cluster_anchored: X = {X.shape}")
        batch_size, bag_size = X.shape[0], X.shape[1]
        with torch.no_grad():
            mask = torch.zeros(batch_size, bag_size, dtype=torch.bool, device=self.device) # True marks already selected elements
            labels = torch.zeros(batch_size, bag_size, dtype=torch.long, device=self.device) # cluster 0 stores unclustered elements
            for i in range(1, max_iter+1):
                # print(f"=============================================================")
                # print(f"DACLightning.cluster_anchored: i = {i}")

                anchor_idxs = self.sample_anchor_idxs(batch_size, bag_size, mask=mask)
                # print(f"DACLightning.cluster_anchored: anchor_idx = {anchor_idxs}")
                
                logits = self.forward(X, anchor_idxs, mask)
                # print(f"DACLightning.cluster_anchored: logits = {logits.shape}")

                select = logits > 0.0
                # print(f"DACLightning.cluster_anchored: select = {select}")

                labels[select*mask.bitwise_not()] = i
                mask[select] = True
                # print(f"DACLightning.cluster_anchored: labels = {labels}")
                # print(f"DACLightning.cluster_anchored: mask = {mask}")

                num_processed = mask.sum(1)
                # print(f"DACLightning.cluster_anchored: num_processed = {num_processed}")
                done = num_processed == bag_size
                # print(f"DACLightning.cluster_anchored: done = {done}")
                if done.all():
                    break

            # print(f"DACLightning.cluster_anchored: FINAL labels = {labels}")
            return labels

    def validation_epoch_end(self, validation_step_outputs):
        losses = AverageMeter()
        ari = AverageMeter()
        nmi = AverageMeter()
        nclusters = AverageMeter()
        for out in validation_step_outputs:
            losses.update(out["loss"])
            targets = out["targets"].cpu()
            plabels = out["plabels"].cpu()
            # print(f"DACLightning.validation_epoch_end: targets = {targets}")
            # print(f"DACLightning.validation_epoch_end: plabels = {plabels}")

            for i in range(targets.shape[0]):
                ari.update(adjusted_rand_score(targets[i], plabels[i]))
                nmi.update(normalized_mutual_info_score(
                    targets[i], plabels[i], average_method='arithmetic'))
                nclusters.update(len(set(plabels[i].numpy())))

        self.log("val_loss", losses.avg, prog_bar=False,
                 logger=True, on_step=False, on_epoch=True)
        self.log("val_ari", ari.avg, prog_bar=False,
                 logger=True, on_step=False, on_epoch=True)
        self.log("val_nmi", nmi.avg, prog_bar=False,
                 logger=True, on_step=False, on_epoch=True)
        self.log("val_nclusters", nclusters.avg, prog_bar=False,
                 logger=True, on_step=False, on_epoch=True)

        # self.log("val_loss", loss, prog_bar=False, logger=True, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.args.lr)
        if self.schedule_fn is None:
            return optimizer
        else:
            scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, self.schedule_fn)
            return [optimizer], [scheduler]

## Schedule fn

In [10]:
def make_schedule_fn(warmup_epochs, stable_epochs, decreasing_epochs):
    def fn(epoch):
        if epoch < warmup_epochs:
            return epoch/warmup_epochs
        elif epoch < warmup_epochs + stable_epochs:
            return 1
        else:
            now_epoch = epoch - stable_epochs - warmup_epochs
            return 1 - now_epoch / decreasing_epochs
    return fn

## Validate

In [15]:
train_loader, eval_loader = OmniglotL2C(batch_size=100, bag_size=400, cls_per_bag=40, num_workers=4, seed=None, test_seed=1234)

args = ArgsOmniglotDAC()
vggs = VGGS(args.feature_size)
schedule_fn = make_schedule_fn(0, 1000, 1000)
dacmodel = DACLightning(args=args).load_from_checkpoint(root_dir / 'DAC_omniglot_400', 
                                                        args=args, featuremodel=vggs,
                                                        schedule_fn=schedule_fn)
pl.Trainer(gpus=1).validate(dacmodel, eval_loader)

Files already downloaded and verified
n_batch_elements: 40000
Files already downloaded and verified


Multiprocessing is handled by SLURM.
/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_ari           0.041308145970106125
        val_loss            0.05387571081519127
      val_nclusters               50.9375
         val_nmi            0.5612772174215839
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.05387571081519127,
  'val_ari': 0.041308145970106125,
  'val_nmi': 0.5612772174215839,
  'val_nclusters': 50.9375}]

## Finetune

In [16]:
train_loader, eval_loader = OmniglotL2C(batch_size=24, bag_size=400, cls_per_bag=40, num_workers=4, seed=None, test_seed=1234)

args = ArgsOmniglotDAC()

vggs = VGGS(args.feature_size)

schedule_fn = make_schedule_fn(100, 100, 100)

dacmodel = DACLightning(args=args).load_from_checkpoint(root_dir / 'DAC_omniglot_bck', 
                                                        args=args, featuremodel=vggs,
                                                        schedule_fn=schedule_fn)

lr_monitor = LearningRateMonitor(logging_interval='step')
# wandb.init(project="amortized_omniglot", name='omniglot400')
# wandb_logger = WandbLogger()

trainer = pl.Trainer(gpus=1, log_every_n_steps=1, max_epochs=610,
                     default_root_dir=root_dir, check_val_every_n_epoch=10,
                     # logger=wandb_logger,
                     callbacks=[lr_monitor]
                    )
trainer.fit(dacmodel, train_loader, eval_loader)

Files already downloaded and verified
n_batch_elements: 9600
Files already downloaded and verified


Multiprocessing is handled by SLURM.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /mnt/data/factcheck/tweets/clustering/lightning_logs/version_4109426/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type     | Params
------------------------------------------
0 | featuremodel | VGG      | 2.0 M 
1 | dacmodel     | DACModel | 3.0 M 
------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
20.015    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7ff4609bf4c0>
Traceback (most recent call last):
  File "/mnt/appl/software/Python/3.9.5-GCCcore-10.3.0/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 2753567, 2753695, 2753823) exited unexpectedly

In [49]:
for t in train_loader:
    break
t[0].shape, t[1].shape

(torch.Size([24, 400, 1, 32, 32]), torch.Size([24, 400]))

In [50]:
images, labels = t
print(labels[0].unique(sorted=True))
print(labels[1].unique(sorted=True))

tensor([ 61,  77,  96, 134, 173, 188, 198, 241, 304, 316, 381, 392, 399, 417,
        419, 463, 468, 491, 494, 520, 538, 546, 559, 630, 631, 668, 747, 751,
        755, 799, 813, 843, 846, 854, 883, 910, 920, 937, 943, 961])
tensor([ 66,  99, 107, 111, 122, 132, 148, 149, 175, 202, 210, 218, 237, 299,
        317, 356, 378, 381, 386, 391, 395, 430, 485, 536, 551, 556, 588, 611,
        624, 709, 735, 803, 815, 825, 846, 876, 887, 890, 928, 936])
